## ccdproc-07: Combining single observations 

<pre> Máster en Astrofísica UCM -- Técnicas Experimentales en Astrofísica
Jaime Zamorano, Nicolás Cardiel and Sergio Pascual

v0  2020/05/12 to be completed
v1  2021/01/18 from example_04_combining_images_v1.ipynb but using NOT2008 observations

</pre>

Note that the ``astropy`` package should be installed. In this sense, have a look to the
astropy installation description: https://docs.astropy.org/en/stable/install.html.
We are also using ``ccdproc`` package.

In [ ]:
from astropy.io import fits
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from ccdproc import CCDData, Combiner

In [ ]:
plt.style.use('./tea.mplstyle')   # Some parameters for nicer graphs

In [ ]:
# auxiliary function to display a rectangle and compute mean value within it
def draw_rectangle(ax, image_data, x1, x2, y1, y2, color, text=False):
    ax.plot((x1, x1), (y1, y2), color, lw=1)
    ax.plot((x2, x2), (y1, y2), color, lw=1)
    ax.plot((x1, x2), (y1, y1), color, lw=1)
    ax.plot((x1, x2), (y2, y2), color, lw=1)
    if text:
        media = image_data[y1:y2,x1:x2].mean()
        std   = image_data[y1:y2,x1:x2].std()
        ax.text((x1+x2)/2, y1+(y2-y1)/8, str(int(media)), 
                ha='center', va='center', color=color, fontsize=12)        
        ax.text((x1+x2)/2, y2-(y2-y1)/8, str(round(std,1)), 
                ha='center', va='top', color=color, fontsize=12)
    return media, std

### Opening the FITS files to be combined

**List of files to be combined**

These are three consecutive images of NGC4941 in R band taken with ALFOSC at NOT 2008

In [ ]:
directory='N1/'
#directory='/Users/jzamorano/Desktop/NOT_2008/N1/'
files = ['120110','120111','120112']       # fzt_ALrd+files[i]

Opening the FITS files and putting the data into a list of numpy 2-D arrays.

In [ ]:
image = []
for i in range(len(files)):
    image.append(fits.open(directory+'fzt_ALrd'+str(files[i])+'.fits')[0])
for i in range(len(files)):
#print(image[0].info())
#print(image[0].header)
    print(image[i].header['FILENAME'],image[i].header['OBJECT'],image[i].header['EXPTIME'])

### Analysing the images data

Let us get some basic statistics about our images

#### Plotting a histogram

To make a histogram with ``matplotlib.pyplot.hist()``, we need to cast the data from a 2-D to array to something one dimensional.
In this case, we are am using ``ndarray.flatten()`` to return a 1-D numpy array.

In [ ]:
plt.xlim(800,2500)
bins = np.arange(800,2500,50)
for i in range(3):
    plt.hist(image[i].data.flatten(), alpha=0.6, bins=bins,label=i)
plt.legend()
plt.show()

 One of the files (the first observation) has more signal than the next ones. 

### Display the images using frame coords (pixels)  
Let display the images with the same background and foreground limit values.

In [ ]:
sky_mean , std = [] , []
fig, axarr = plt.subplots(ncols=3, nrows=1, figsize=(12, 12))
for n in range(3):
    ax = axarr[n]
    ax.imshow(image[n].data, cmap='gray', origin='lower',vmin=1000, vmax=5000,norm=LogNorm())
    ax.text(200,200,n,fontsize=15,color="w")
    ax.set_xlabel('X axis')
    mean_n,std_n = draw_rectangle(ax, image[n].data, 100, 500, 1500, 1900, color='w',text=True)
    print(n,int(mean_n),int(std_n))
    sky_mean.append(mean_n)
    std.append(std_n)
    ax.grid()
    

There are differences in sky background, but also un the telescope pointing. A small offset was applied before each observation.  

### Display the images using WCS coordinates (RA & DEC)  
Let display the images using the astrometry information of the headers 

In [ ]:
from astropy.wcs import WCS

In [ ]:
fig = plt.subplots(figsize=(14, 12))
for n in range(3):
    ax1 = plt.subplot(1,3,n+1, projection=WCS(headers[n]))
    ax1.imshow(image[n].data, origin='lower', vmin=1000., vmax=5000.)
    ax1.coords['ra'].set_axislabel('Right Ascension')
    ax1.coords['dec'].set_axislabel('Declination')
    plt.grid(color='w')
    ax1.set_title(headers[n]['FILENAME'])

In [ ]:
Using WCS coordinates the images are aligned.

### Using ``reproject```

We will use ``reproject``  
https://reproject.readthedocs.io/en/stable/ 
"The reproject package implements image reprojection (resampling) methods for astronomical images and more generally n-dimensional data. These assume that the WCS information contained in the data are correct."

In [ ]:
from reproject import reproject_interp

In [ ]:
n_image_1, footprint = reproject_interp(image[1], image[0].header)
n_image_2, footprint = reproject_interp(image[2], image[0].header)

In [ ]:
fig = plt.subplots(figsize=(14, 12))

ax1 = plt.subplot(1,3,1, projection=WCS(headers[0]))
ax1.imshow(image[0].data, origin='lower', vmin=1000., vmax=5000.)
ax1.coords['ra'].set_axislabel('Right Ascension')
ax1.coords['dec'].set_axislabel('Declination')
plt.grid(color='w')
ax1.set_title(headers[0]['FILENAME'])

ax1 = plt.subplot(1,3,2, projection=WCS(headers[0]))
ax1.imshow(n_image_1, origin='lower', vmin=1000., vmax=5000.)
ax1.coords['ra'].set_axislabel('Right Ascension')
ax1.coords['dec'].set_axislabel('Declination')
plt.grid(color='w')
ax1.set_title(headers[1]['FILENAME'])

ax1 = plt.subplot(1,3,3, projection=WCS(headers[0]))
ax1.imshow(n_image_2, origin='lower', vmin=1000., vmax=5000.)
ax1.coords['ra'].set_axislabel('Right Ascension')
ax1.coords['dec'].set_axislabel('Declination')
plt.grid(color='w')
ax1.set_title(headers[1]['FILENAME'])

The white stripes of the second (at left) and third images (left and top)are the area of the sky in the FoV of the first observation and not covered in that observations.

In [ ]:
nimage_1 = image[1].copy()
nimage_1.data = n_image_1
nimage_1.header = image[0].header
nimage_1.header['FILENAME'] = 'nfzt_ALrd120111.fits' 
nimage_1.header['HISTORY'] = 'projected to fzt_ALrd120111.fits'
nimage_1.writeto(directory+'nfzt_ALrd120111.fits',overwrite='yes')

In [ ]:
nimage_2 = image[2].copy()
nimage_2.data = n_image_2
nimage_2.header = image[0].header
nimage_2.header['FILENAME'] = 'nfzt_ALrd120112.fits' 
nimage_2.header['HISTORY'] = 'projected to fzt_ALrd120110.fits'
nimage_2.writeto(directory+'nfzt_ALrd120112.fits',overwrite='yes')

### Combining the individual images

We can read the FITS files created as CCDDAta objects to create the ``Combiner``

In [ ]:
ccd = []
ccd.append(CCDData.read(directory+'fzt_ALrd120110.fits')) 
ccd.append(CCDData.read(directory+'nfzt_ALrd120111.fits',unit='adu')) 
ccd.append(CCDData.read(directory+'nfzt_ALrd120112.fits')) 

In [ ]:
sky_mean , std = [] , []
fig, axarr = plt.subplots(ncols=3, nrows=1, figsize=(12, 12))
for n in range(3):
    ax = axarr[n]
    ax.imshow(ccd[n].data, cmap='gray', origin='lower',vmin=1000, vmax=5000,norm=LogNorm())
    ax.text(200,200,n,fontsize=15,color="w")
    ax.set_xlabel('X axis')
    mean_n,std_n = draw_rectangle(ax, ccd[n].data, 100, 500, 1500, 1900, color='w',text=True)
    print(n,int(mean_n),int(std_n))
    sky_mean.append(mean_n)
    std.append(std_n)
    ax.grid()
    

The images are aligned. Let zoom in for a detailed view. 

In [ ]:
sky_mean , std = [] , []
fig, axarr = plt.subplots(ncols=3, nrows=1, figsize=(12, 12))
for n in range(3):
    ax = axarr[n]
    ax.imshow(ccd[n].data[900:1450,950:1300], cmap='gray', origin='lower',vmin=1300, vmax=2000,norm=LogNorm())
    ax.minorticks_on()
    ax.grid(True,which='major',color='w',linestyle='-',lw=0.5)
    ax.grid(True,which='minor',color='w',linestyle='--',lw=0.2)
    

The alignnement is not perfect probably because the astrometry was made on the fly during the observations.

As the images have different signal we must scale the signal to combine the individual images
https://ccdproc.readthedocs.io/en/latest/api/ccdproc.Combiner.html#ccdproc.Combiner.scaling

We will use a part of the images with signal to avoid the empty pixels qith nan values

In [ ]:
combiner = Combiner(ccd)
scaling_func = lambda arr: 1400/np.ma.average(arr[500:1000,1500:2000])
combiner.scaling = scaling_func 

In [ ]:
print(combiner.scaling)

In [ ]:
# median combine 
combined_image_average_scaled = combiner.average_combine()
# median combine 
combined_image_median = combiner.median_combine()

In [ ]:
fig = plt.figure(figsize=(14, 12))
ax0 = fig.add_subplot(131)
box = combined_image_average_scaled.data
img = ax0.imshow(box, cmap='gray', origin='lower',vmin=1300, vmax=2000,norm=LogNorm())
ax0.grid(color='w')
ax1 = fig.add_subplot(132)
box = combined_image_average_scaled.data[1200:1500,1000:1300]
img = ax1.imshow(box, cmap='gray', origin='lower',vmin=1300, vmax=2000,norm=LogNorm())
ax1.grid(color='w')
ax2 = fig.add_subplot(133)
box = ccd[0].data[1200:1500,1000:1300]
img = ax2.imshow(box, cmap='gray', origin='lower',vmin=1300, vmax=2000,norm=LogNorm())
ax2.grid(color='w')

### Writing the resulting file

We can use the ``writeto`` method to save the new file. 
Note that ``writeto`` will close the new file for you.

In [ ]:
# Extract primary header from single file and copy into new combined file header
HDUList_object = ccd[0].header
combined_image_median.header = HDUList_object

In [ ]:
# Replace FILENAME keyword and add information
combined_image_median.header['FILENAME'] = 'dummy.fits' 
combined_image_median.header['HISTORY'] = 'median combining '+ str(len(files)) + 'images'
combined_image_median.header['HISTORY'] = 'fztALrd120110 nfzt_ALrd120111 nfzt_ALrd120112'


In [ ]:
# Finally writte the image to a FITS file. 
# An error occurs when the file already exits (use overwrite='yes' to avoid the error)
combined_image_median.write(directory+'dummy.fits',overwrite='yes')